use the items to create relations. 

for eg: a table for license and owners, etc. 

eg table: owner, licence, topics and repositories. 

sample questions: 
- how many repo facebook research is using?
- what is the repo with most starts?
- what are the projs with apache 2 license. 
- how many stars does facebook research have?



In [109]:
# Get data
import requests
response = requests.get("https://api.github.com/search/repositories?q=created:>2023-01-01+language:python&sort=stars&order=desc&per_page=100")

if response.status_code == 200:
    data_js = response.json()
    print(f"Request successful. Response items size: {len(data_js['items'])}")
else:
    print("Something went wrong with our request:", response.text)

Request successful. Response items size: 100


In [110]:
import os
import json

items = data_js['items']

for item in items:
    directory = "repositories/" + item['full_name']
    
    if not os.path.exists(directory):
        os.makedirs(directory)

    metadata_path = directory + "/metadata.json"

    with open(metadata_path, "w") as f:
        json.dump({"repository_name": item["name"],
            "html_url": item["html_url"],
            "owner": item["owner"]["login"]}, f)

In [111]:
owners = []

for item in items:
    for key in list(item.keys()):
        if "url" in key and key != "url":
            del item[key]
        if "owner" == key:
            owner = item[key]
            for keyo in list(owner.keys()):
                if "url" in keyo and keyo != "url":
                    del owner[keyo]
            owners.append(owner)
            del item[key]

In [112]:
print(items[3])

{'id': 600798098, 'node_id': 'R_kgDOI89zkg', 'name': 'sd-webui-controlnet', 'full_name': 'Mikubill/sd-webui-controlnet', 'private': False, 'description': 'WebUI extension for ControlNet', 'fork': False, 'url': 'https://api.github.com/repos/Mikubill/sd-webui-controlnet', 'created_at': '2023-02-12T16:26:27Z', 'updated_at': '2023-03-06T19:46:57Z', 'pushed_at': '2023-03-06T16:59:58Z', 'homepage': '', 'size': 8418, 'stargazers_count': 4105, 'watchers_count': 4105, 'language': 'Python', 'has_issues': True, 'has_projects': True, 'has_downloads': True, 'has_wiki': True, 'has_pages': False, 'has_discussions': True, 'forks_count': 401, 'archived': False, 'disabled': False, 'open_issues_count': 143, 'license': {'key': 'mit', 'name': 'MIT License', 'spdx_id': 'MIT', 'url': 'https://api.github.com/licenses/mit', 'node_id': 'MDc6TGljZW5zZTEz'}, 'allow_forking': True, 'is_template': False, 'web_commit_signoff_required': False, 'topics': [], 'visibility': 'public', 'forks': 401, 'open_issues': 143, 'w

In [113]:
import pandas as pd 

pd.DataFrame.from_dict(items).head(2)

,id,node_id,name,full_name,private,description,fork,url,created_at,updated_at,...,allow_forking,is_template,web_commit_signoff_required,topics,visibility,forks,open_issues,watchers,default_branch,score
0,595893961,R_kgDOI4SeyQ,ControlNet,lllyasviel/ControlNet,False,Let us control diffusion models!,False,https://api.github.com/repos/lllyasviel/Contro...,2023-02-01T02:43:30Z,2023-03-06T19:40:45Z,...,True,False,False,[],public,938,129,11888,main,1.0
1,601538369,R_kgDOI9q_QQ,llama,facebookresearch/llama,False,Inference code for LLaMA models,False,https://api.github.com/repos/facebookresearch/...,2023-02-14T09:29:12Z,2023-03-06T19:49:49Z,...,True,False,False,[],public,979,104,7795,main,1.0


In [125]:
df = pd.DataFrame.from_dict(data_js['items'])
owners_df = pd.DataFrame.from_dict(owners) \
                        .drop_duplicates() \
                        .reset_index(drop=True)

In [126]:
# licence_df = df['license'].apply(pd.Series)
# licence_df = licence_df.drop_duplicates()

licence_df = pd.json_normalize(df['license']) \
               .drop_duplicates() \
               .reset_index(drop=True)

In [127]:
display(owners_df.sample(3))
display(licence_df.sample(3))


,login,id,node_id,gravatar_id,url,type,site_admin
5,BlinkDL,33809201,MDQ6VXNlcjMzODA5MjAx,,https://api.github.com/users/BlinkDL,User,False
54,maxi1134,1342643,MDQ6VXNlcjEzNDI2NDM=,,https://api.github.com/users/maxi1134,User,False
85,ashawkey,25863658,MDQ6VXNlcjI1ODYzNjU4,,https://api.github.com/users/ashawkey,User,False


,key,name,spdx_id,url,node_id
3,other,Other,NOASSERTION,None,MDc6TGljZW5zZTA=
7,bsd-2-clause,"BSD 2-Clause ""Simplified"" License",BSD-2-Clause,https://api.github.com/licenses/bsd-2-clause,MDc6TGljZW5zZTQ=
6,cc-by-sa-4.0,Creative Commons Attribution Share Alike 4.0 I...,CC-BY-SA-4.0,https://api.github.com/licenses/cc-by-sa-4.0,MDc6TGljZW5zZTI2
